# DB 化

フィールドの説明は [ここ](https://open.fda.gov/apis/drug/label/searchable-fields/) から見られる。  
20000 行で 600 MB とかっぽい。5000 行ごとに区切る。

In [ ]:
import pandas as pd
import sqlite3
import os

csv_file = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_columns.csv"  # カラム情報
columns_info = pd.read_csv(csv_file)

# TSVファイルが格納されているディレクトリ
tsv_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_raw_df/"

chunk_index = 1

tsv_files = [f for f in os.listdir(tsv_dir) if f.endswith(".tsv")]
tsv_files.sort()

# 通し番号（row_id）の初期値
last_row_id = 1

# ディレクトリ内のすべてのTSVファイルを処理
for tsv_file in tsv_files:
    if tsv_file.endswith(".tsv"):
        tsv_file_path = os.path.join(tsv_dir, tsv_file)
        
        tsv_data = pd.read_csv(tsv_file_path, sep='\t', low_memory=False)

        # 除外しないカラムを選択
        included_columns = columns_info[(columns_info['除外する'] == False)]['DLしたデータの列'].values
        exist_columns = [col for col in included_columns if col in tsv_data.columns]

        # data から除外しないカラムを抽出
        tsv_data_filtered = tsv_data[exist_columns]

        # CSVにあるがTSVにない列を追加（欠損値で埋める）
        tsv_data_filtered = tsv_data_filtered.copy()
        missing_columns = set(included_columns) - set(tsv_data_filtered.columns)
        for col in missing_columns:
            tsv_data_filtered[col] = None

        # 列の順番をCSVに基づいて整理
        tsv_data_filtered = tsv_data_filtered[included_columns]

        # 通し番号（ID）を追加（ファイルを跨いで連続）
        tsv_data_filtered.insert(0, 'row_id', range(last_row_id, last_row_id + len(tsv_data_filtered)))
        last_row_id += len(tsv_data_filtered)

        chunk_size = 5000  # 5000行ごとに分割
        num_rows = len(tsv_data_filtered)
        num_chunks = (num_rows // chunk_size) + 1
        base_db_file = "fdalabel"  # DBファイル名
        table_name = "raw"

        chunk_start = 0

        while chunk_start < num_rows:
            db_file = f"/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_db/{base_db_file}_{chunk_index:04d}.db"
            conn = sqlite3.connect(db_file)
            cursor = conn.cursor()

            # テーブルを作る
            columns_sql = "row_id INT, " + ", ".join([f'"{col}" TEXT' for col in tsv_data_filtered.columns if col != 'row_id'])
            create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"
            cursor.execute(create_table_sql)

            # 分割
            chunk_end = min(chunk_start + chunk_size, num_rows)
            chunk = tsv_data_filtered.iloc[chunk_start:chunk_end]

            # Insert
            column_names = "row_id, " + ", ".join([f'"{col}"' for col in chunk.columns if col != 'row_id'])
            placeholders = ", ".join(["?" for _ in chunk.columns])
            insert_sql = f'INSERT INTO "{table_name}" ({column_names}) VALUES ({placeholders});'

            cursor.executemany(insert_sql, chunk.values.tolist())
            conn.commit()

            chunk_start = chunk_end
            chunk_index += 1

            conn.close()

In [6]:
csv_columns = columns_info['DLしたデータの列'].values
tsv_columns = tsv_data.columns.tolist()

# 一致するカラムを表示
common_columns = [col for col in csv_columns if col in tsv_columns]
print("CSVとTSVで一致するカラム:", common_columns)

# 一致しないカラムを表示
missing_in_tsv = [col for col in csv_columns if col not in tsv_columns]
print("TSVに存在しないCSVカラム:", missing_in_tsv)

CSVとTSVで一致するカラム: ['set_id', 'id', 'abuse', 'abuse_table', 'accessories', 'active_ingredient', 'active_ingredient_table', 'adverse_reactions', 'adverse_reactions_table', 'alarms', 'alarms_table', 'animal_pharmacology_and_or_toxicology', 'animal_pharmacology_and_or_toxicology_table', 'ask_doctor', 'ask_doctor_or_pharmacist', 'assembly_or_installation_instructions', 'boxed_warning', 'boxed_warning_table', 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility', 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility_table', 'cleaning', 'clinical_pharmacology', 'clinical_pharmacology_table', 'clinical_studies', 'clinical_studies_table', 'components', 'contraindications', 'contraindications_table', 'controlled_substance', 'dependence', 'description', 'description_table', 'disposal_and_waste_handling', 'do_not_use', 'do_not_use_table', 'dosage_and_administration', 'dosage_and_administration_table', 'dosage_forms_and_strengths', 'dosage_forms_and_strengths_table', 'drug_abuse_and_dep